In [ ]:
!pip install faker
!pip install randomtimestamp

In [ ]:
# @title Подключение библиотек
import sqlite3
import os
from faker import Faker
from random import randint
from randomtimestamp import randomtimestamp
from sqlalchemy import UUID, Column, String, Integer, ForeignKey, Float, Date, select, func, DDL, event
from typing import List
from sqlalchemy.sql import text
from sqlalchemy.orm import relationship, Mapped, declarative_base, sessionmaker
from sqlalchemy.engine import URL, create_engine
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime, timedelta

CATEGORY_NAMES = ["Дети", "Подростки", "Студенты", "Семейная пара", "Пенсионеры", "Инвалиды", "Граждане"]
MAINTENANCES_TYPES = ["Regular cleaning", "repair", "chemical treatment", 'filter replacement']
SERVICES_NAMES = ["Аквааэробика", "Прокат нарукавников", "Прокат круга для плавания", "Прокат шапочки для плавания", "Прокат спасательного жилета"]

In [ ]:
# @title Создание базы
engine = create_engine("sqlite:///database.db")
Base = declarative_base()

In [ ]:
# @title Создание моделей
class Category(Base):
    __tablename__ = 'categories'

    category_ID = Column(Integer, primary_key=True)
    category_name = Column(String, nullable=False)
    category_cost = Column(Integer, nullable=False)

    subscriptions = relationship('Subscription', back_populates='category')

class Clients(Base):
    __tablename__ = 'clients'

    client_ID = Column(Integer, primary_key=True)
    client_name = Column(String, nullable=False)
    client_birthdate = Column(Date, nullable=False)
    client_phone_number = Column(String, nullable=False)

    subscriptions = relationship('Subscription', back_populates='client')

class Coach(Base):
    __tablename__ = 'coaches'

    coach_ID = Column(Integer, primary_key=True)
    coach_name = Column(String, nullable=False)

    group_lessons = relationship('GroupLesson', back_populates='coach')

class Lesson(Base):
    __tablename__ = 'lessons'

    lesson_ID = Column(Integer, primary_key=True)

    subscription_ID = Column(Integer, ForeignKey('subscriptions.subscription_ID'))
    pool_room_id = Column(Integer, ForeignKey('pool_rooms.pool_room_ID'))

    lesson_date = Column(Date, nullable=False)
    visiting_time = Column(String, nullable=False)
    track_num = Column(Integer, nullable=False)

    group_lessons = relationship('GroupLesson', back_populates='lesson')
    subscription = relationship('Subscription', back_populates='lessons')
    pool_room = relationship('PoolRoom', back_populates='lessons')
    lesson_services = relationship('LessonService', back_populates='lesson')

class GroupLesson(Base):
    __tablename__ = 'group_lessons'

    group_lesson_ID = Column(Integer, ForeignKey('lessons.lesson_ID'), primary_key=True)
    coach_id = Column(Integer, ForeignKey('coaches.coach_ID'), primary_key=True)

    lesson = relationship('Lesson', back_populates='group_lessons')
    coach = relationship('Coach', back_populates='group_lessons')

class LessonService(Base):
    __tablename__ = 'lesson_services'

    service_ID = Column(Integer, ForeignKey('services.service_ID'), primary_key=True)
    lesson_ID = Column(Integer, ForeignKey('lessons.lesson_ID'), primary_key=True)

    services = relationship('Service', back_populates='lesson_services')
    lesson = relationship('Lesson', back_populates='lesson_services')

class Maintenance(Base):
    __tablename__ = 'maintenance'
    maintenances_ID = Column(Integer, primary_key=True)
    maintenance_date = Column(Date, nullable=False)
    maintenance_type = Column(String, nullable=False)
    pool_room_ID = Column(Integer, ForeignKey('pool_rooms.pool_room_ID'))

    pool_room = relationship('PoolRoom', back_populates='maintenances')

class PoolRoom(Base):
    __tablename__ = 'pool_rooms'

    pool_room_ID = Column(Integer, primary_key=True)
    quantity_tracks = Column(Integer, nullable=False)

    lessons = relationship('Lesson', back_populates='pool_room')
    maintenances = relationship('Maintenance', back_populates='pool_room')

class Service(Base):
    __tablename__ = 'services'

    service_ID = Column(Integer, primary_key=True)
    service_name = Column(String, nullable=False)
    service_price = Column(Integer, nullable=False)
    lesson_services = relationship('LessonService', back_populates='services')

class Subscription(Base):
    __tablename__ = 'subscriptions'

    subscription_ID = Column(Integer, primary_key=True)
    subscription_start_day = Column(Date, nullable=False)
    subscription_end_day = Column(Date, nullable=False)
    client_ID = Column(Integer, ForeignKey('clients.client_ID'))
    category_ID = Column(Integer, ForeignKey('categories.category_ID'))

    client = relationship('Clients', back_populates='subscriptions')
    category = relationship('Category', back_populates='subscriptions')
    lessons = relationship('Lesson', back_populates='subscription')


In [ ]:
# @title Создание таблиц в соответствии с моделями
Base.metadata.create_all(bind=engine)
Session = sessionmaker(autoflush=False, bind=engine)

In [ ]:
# @title Генерация данных
fake = Faker()
categories = [Category(category_name=category, category_cost = randint(100, 300)) for category in CATEGORY_NAMES]
maintenances = [Maintenance(maintenance_date = randomtimestamp(text=False), maintenance_type = maintenance, pool_room_ID = randint(1,2)) for maintenance in MAINTENANCES_TYPES]
coaches = [Coach(coach_name = fake.name()) for _ in range(10)]
clients = [Clients(client_name = fake.name(), client_birthdate = datetime.strptime(fake.date(), '%Y-%m-%d'), client_phone_number = fake.phone_number())]
pool_rooms = [PoolRoom(quantity_tracks = randint(1,3)) for _ in range(4)]
services = [Service(service_name = service, service_price = randint(130,300)) for service in SERVICES_NAMES]
subscriptions = [Subscription(subscription_start_day = (start_day := datetime.strptime(fake.date(), '%Y-%m-%d')), subscription_end_day = start_day + timedelta(days=30), client_ID = i, category_ID = randint(1, 7)) for i in range(10)]
lessons = [Lesson(subscription_ID = i, pool_room_id = randint(1,4), lesson_date = datetime.strptime(fake.date(), '%Y-%m-%d'), visiting_time = '16:00 - 17:00', track_num = randint(1,5)) for i in range(10)]
group_lessons = [GroupLesson(group_lesson_ID = randint(1,10), coach_id = randint(1, 10)) for i in range(4)]
lesson_services = [LessonService(service_ID = randint(1,len(SERVICES_NAMES)), lesson_ID = randint(1,10)) for i in range(4)]

In [ ]:
# @title Добавление
with Session(autoflush=False, bind=engine) as session:
  try:
    session.add_all(categories)
    session.add_all(maintenances)
    session.add_all(coaches)
    session.add_all(clients)
    session.add_all(pool_rooms)
    session.add_all(services)
    session.add_all(subscriptions)
    session.add_all(lessons)
    session.add_all(group_lessons)
    session.add_all(lesson_services)
    session.commit()
  except SQLAlchemyError as ex:
    session.rollback()
    print(f"Возникла ошибка: {ex}")

In [ ]:
# @title Чтение
with Session(autoflush = False, bind = engine) as session:
  maintenances_ = session.query(Maintenance).all()
  for maintenance in maintenances_:
    print(maintenance.maintenances_ID, maintenance.maintenance_date, maintenance.maintenance_type)

1 1976-12-25 Regular cleaning
3 1989-05-02 Пример Тип обслуживания
4 2015-06-15 filter replacement
5 1991-07-20 Regular cleaning
6 1979-03-15 repair
7 1963-07-08 chemical treatment
8 1990-01-21 filter replacement
9 1972-06-17 Regular cleaning
10 1988-01-06 repair
11 1956-03-09 chemical treatment
12 1951-05-26 filter replacement
13 1983-10-01 Regular cleaning
14 1998-05-03 repair
15 1950-03-07 chemical treatment
16 2000-02-25 filter replacement


In [ ]:
# @title Обновление
with Session(autoflush = False, bind = engine) as session:
  maintenance = session.query(Maintenance).get(3)
  maintenance.maintenance_type = "Пример Тип обслуживания"
  session.commit()

<ipython-input-54-f46f0da27d63>:2: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  maintenance = session.query(Maintenance).get(3)


In [ ]:
# @title Удаление
with Session(autoflush = False, bind = engine) as session:
  maintenance = session.query(Maintenance).get(2)
  session.delete(maintenance)
  session.commit()

<ipython-input-56-b4c04881e620>:2: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  maintenance = session.query(Maintenance).get(2)


In [ ]:
# @title Сложный запрос
# формируем запрос
query = session.query(
    Subscription.client_ID,
    func.count(Subscription.client_ID).label('total')
).join(
    Clients, Subscription.client_ID == Clients.client_ID
).group_by(
    Subscription.client_ID
).having(
    func.count(Subscription.client_ID) >= 2
).order_by(
    Subscription.client_ID
)

results = query.all()
for result in results:
  print(result)

session.close()

(1, 2)
(2, 2)
(3, 2)


In [ ]:
# @title Транзакция
with Session(autoflush = False, bind = engine) as session:
  session.begin_nested()
  try:
      client = Clients(client_name = "Robert Polsen", client_birthdate = datetime(year = 2000, month = 1, day = 1), client_phone_number = fake.phone_number())
      session.add(client)
      session.flush()
      print('Запись изменена')
  except SQLAlchemyError:
      session.rollback()
      print('Запись не была изменена')

Запись изменена


In [ ]:
# @title Триггер
trigger_request = DDL('''\
CREATE TRIGGER change_subscriptions
AFTER INSERT OR UPDATE ON subscriptions
BEGIN
     SELECT 'Добавленная или измененная запись: ID:' || NEW.subscription_ID || ,'\nДата начала абонемента = ' || NEW.subscriptions.subscription_start_day || ',\nДата конца абонемента = ' || NEW.subscriptions.subscription_end_day
END;
''')
event.listen(Subscription.__table__, 'after_create', trigger_request)

<ipython-input-92-8eb84f49abad>:12: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  subscription = session.query(Subscription).get(1)


In [ ]:
# @title Представление
import sqlalchemy as sa
from sqlalchemy.ext import compiler
from sqlalchemy.schema import DDLElement
from sqlalchemy.sql import table


class CreateView(DDLElement):
    def __init__(self, name, selectable):
        self.name = name
        self.selectable = selectable


class DropView(DDLElement):
    def __init__(self, name):
        self.name = name


@compiler.compiles(CreateView)
def _create_view(element, compiler, **kw):
    return "CREATE VIEW %s AS %s" % (
        element.name,
        compiler.sql_compiler.process(element.selectable, literal_binds=True),
    )


@compiler.compiles(DropView)
def _drop_view(element, compiler, **kw):
    return "DROP VIEW %s" % (element.name)


def view_exists(ddl, target, connection, **kw):
    return ddl.name in sa.inspect(connection).get_view_names()


def view_doesnt_exist(ddl, target, connection, **kw):
    return not view_exists(ddl, target, connection, **kw)


def view(name, metadata, selectable):
    t = table(name)

    t._columns._populate_separate_keys(
        col._make_proxy(t) for col in selectable.selected_columns
    )

    sa.event.listen(
        metadata,
        "after_create",
        CreateView(name, selectable).execute_if(callable_=view_doesnt_exist),
    )
    sa.event.listen(
        metadata, "before_drop", DropView(name).execute_if(callable_=view_exists)
    )
    return t


if __name__ == "__main__":
    metadata = sa.MetaData()

    stuff_view = view(
        "stuff_view",
        metadata,
        sa.select(
            stuff.c.id.label("id"),
            stuff.c.data.label("data"),
            more_stuff.c.data.label("moredata"),
        )
        .select_from(stuff.join(more_stuff))
        .where(stuff.c.data.like(("%orange%"))),
    )

    # illustrate ORM usage
    from sqlalchemy.orm import declarative_base
    from sqlalchemy.orm import Session

    Base = declarative_base(metadata=metadata)

    class MyStuff(Base):
        __table__ = stuff_view

        def __repr__(self):
            return f"MyStuff({self.id!r}, {self.data!r}, {self.moredata!r})"

    with Session(engine) as s:
        print(s.query(MyStuff).all())